In [1]:
# 声明库依赖
import pickle
import numpy as np
import tensorflow as tf

image_size = 28
num_labels = 10

class NotMNIST:
    def __init__(self):
        class Train:
            def __init__(self):
                self.images = []
                self.labels = []
                self.batch_counter = 0

            @property
            def num_examples(self):
                return len(self.images)

            def next_batch(self, num):
                if self.batch_counter + num >= len(self.labels):
                    batch_images = self.images[self.batch_counter:]
                    batch_labels = self.labels[self.batch_counter:]
                    left = num - len(batch_labels)
                    self.batch_counter = left
                else:
                    batch_images = self.images[self.batch_counter:self.batch_counter + num]
                    batch_labels = self.labels[self.batch_counter:self.batch_counter + num]
                    self.batch_counter += num
                return batch_images, batch_labels

        class Test:
            def __init__(self):
                self.images = []
                self.labels = []

        self.train = Train()
        self.test = Test()

        pickle_file = '../data/notMNIST.pickle'

        with open(pickle_file, 'rb') as f:
            save = pickle.load(f)
            train_dataset = save['train_dataset']
            train_labels = save['train_labels']
            valid_dataset = save['valid_dataset']
            valid_labels = save['valid_labels']
            test_dataset = save['test_dataset']
            test_labels = save['test_labels']
            del save  # 删除内存文件，等待gc回收释放内存

        def reformat(dataset, labels):
            dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
            labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
            return dataset, labels

        train_dataset, train_labels = reformat(train_dataset, train_labels)
        valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
        test_dataset, test_labels = reformat(test_dataset, test_labels)
        print('Training set', train_dataset.shape, train_labels.shape)
        print('Validation set', valid_dataset.shape, valid_labels.shape)
        print('Test set', test_dataset.shape, test_labels.shape)
        self.train.images = train_dataset
        self.train.labels = train_labels
        self.test.images = test_dataset
        self.test.labels = test_labels

In [24]:
# 两层神经网络--隐藏层拥有512个神经元

not_mnist = NotMNIST()
# 参数
learning_rate = 0.01  # 梯度下降步长，寻找最优解的下降步长
training_epochs = 25  # 迭代轮数
batch_size = 100  # 批次训练数据集大小
display_step = 1

# tf图输入
x = tf.placeholder(tf.float32, [None, 784])  # mnist 数据集图片大小为28*28=784，placeholder为占位符
y = tf.placeholder(tf.float32, [None, 10])  # 识别A-J的数据，一共10个类别

# 设置模型权重
W1 = tf.Variable(tf.truncated_normal([784, 256], stddev=0.1))
b1 = tf.Variable(tf.zeros([256]))


hiddle = tf.nn.relu(tf.matmul(x, W1) + b1)

w2 = tf.Variable(tf.truncated_normal([256, 10], stddev=0.1))
b2 = tf.Variable(tf.zeros([10]))

# 构造模型
pred = tf.nn.softmax(tf.matmul(hiddle, w2) + b2)  # Softmax

# 损失函数：交叉墒
cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
# 使用梯度下降算法寻找最优解
# API tf.train.GradientDescentOptimizer
# TODO 创建梯度下降优化方法
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
# 初始化所有变量
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# 启动tf图
with tf.Session() as sess:
    sess.run(init)  # 初始化所有变量

    # 迭代训练
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(not_mnist.train.num_examples / batch_size)  # 计算数据集总共有多少批次数据
        # 训练训练所有批次数据
        for i in range(total_batch):
            batch_xs, batch_ys = not_mnist.train.next_batch(batch_size)
            # 运行优化操作和损失函数计算操作，获取损失值
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 计算平均损失
            avg_cost += c / total_batch
        # 打印显示
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # 测试模型
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # 计算准确度
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: not_mnist.test.images, y: not_mnist.test.labels}))
    saver.save(sess, 'my-model')

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)
Epoch: 0001 cost= 0.765619902
Epoch: 0002 cost= 0.597104537
Epoch: 0003 cost= 0.560341704
Epoch: 0004 cost= 0.536839921
Epoch: 0005 cost= 0.518980493
Epoch: 0006 cost= 0.504291618
Epoch: 0007 cost= 0.491700808
Epoch: 0008 cost= 0.480585596
Epoch: 0009 cost= 0.470604560
Epoch: 0010 cost= 0.461521888
Epoch: 0011 cost= 0.453200024
Epoch: 0012 cost= 0.445534964
Epoch: 0013 cost= 0.438440178
Epoch: 0014 cost= 0.431842074
Epoch: 0015 cost= 0.425677857
Epoch: 0016 cost= 0.419879858
Epoch: 0017 cost= 0.414410393
Epoch: 0018 cost= 0.409247176
Epoch: 0019 cost= 0.404340382
Epoch: 0020 cost= 0.399668858
Epoch: 0021 cost= 0.395222976
Epoch: 0022 cost= 0.390960684
Epoch: 0023 cost= 0.386871232
Epoch: 0024 cost= 0.382937162
Epoch: 0025 cost= 0.379145416
Optimization Finished!
Accuracy: 0.9396


In [25]:
# 两层神经网络--隐藏层拥有512个神经元

not_mnist = NotMNIST()
# 参数
learning_rate = 0.01  # 梯度下降步长，寻找最优解的下降步长
training_epochs = 25  # 迭代轮数
batch_size = 100  # 批次训练数据集大小
display_step = 1

# tf图输入
x = tf.placeholder(tf.float32, [None, 784])  # mnist 数据集图片大小为28*28=784，placeholder为占位符
y = tf.placeholder(tf.float32, [None, 10])  # 识别A-J的数据，一共10个类别

# 设置模型权重
W1 = tf.Variable(tf.truncated_normal([784, 512], stddev=0.1))
b1 = tf.Variable(tf.zeros([512]))


hiddle = tf.nn.relu(tf.matmul(x, W1) + b1)

w2 = tf.Variable(tf.truncated_normal([512, 10], stddev=0.1))
b2 = tf.Variable(tf.zeros([10]))

# 构造模型
pred = tf.nn.softmax(tf.matmul(hiddle, w2) + b2)  # Softmax

# 损失函数：交叉墒
cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
# 使用梯度下降算法寻找最优解
# API tf.train.GradientDescentOptimizer
# TODO 创建梯度下降优化方法
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
# 初始化所有变量
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# 启动tf图
with tf.Session() as sess:
    sess.run(init)  # 初始化所有变量

    # 迭代训练
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(not_mnist.train.num_examples / batch_size)  # 计算数据集总共有多少批次数据
        # 训练训练所有批次数据
        for i in range(total_batch):
            batch_xs, batch_ys = not_mnist.train.next_batch(batch_size)
            # 运行优化操作和损失函数计算操作，获取损失值
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 计算平均损失
            avg_cost += c / total_batch
        # 打印显示
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # 测试模型
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # 计算准确度
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: not_mnist.test.images, y: not_mnist.test.labels}))
    saver.save(sess, 'my-model')

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)
Epoch: 0001 cost= 0.724266465
Epoch: 0002 cost= 0.575478733
Epoch: 0003 cost= 0.537664323
Epoch: 0004 cost= 0.512799944
Epoch: 0005 cost= 0.493848365
Epoch: 0006 cost= 0.478331392
Epoch: 0007 cost= 0.465139016
Epoch: 0008 cost= 0.453594499
Epoch: 0009 cost= 0.443310652
Epoch: 0010 cost= 0.434041851
Epoch: 0011 cost= 0.425597490
Epoch: 0012 cost= 0.417830489
Epoch: 0013 cost= 0.410625120
Epoch: 0014 cost= 0.403904663
Epoch: 0015 cost= 0.397598555
Epoch: 0016 cost= 0.391668642
Epoch: 0017 cost= 0.386073805
Epoch: 0018 cost= 0.380769368
Epoch: 0019 cost= 0.375719022
Epoch: 0020 cost= 0.370909802
Epoch: 0021 cost= 0.366296992
Epoch: 0022 cost= 0.361877148
Epoch: 0023 cost= 0.357624546
Epoch: 0024 cost= 0.353539287
Epoch: 0025 cost= 0.349594764
Optimization Finished!
Accuracy: 0.9435


In [27]:
# 三层神经网络

not_mnist = NotMNIST()
# 参数
learning_rate = 0.01  # 梯度下降步长，寻找最优解的下降步长
training_epochs = 25  # 迭代轮数
batch_size = 100  # 批次训练数据集大小
display_step = 1

# tf图输入
x = tf.placeholder(tf.float32, [None, 784])  # mnist 数据集图片大小为28*28=784，placeholder为占位符
y = tf.placeholder(tf.float32, [None, 10])  # 识别A-J的数据，一共10个类别

# 设置模型权重
W1 = tf.Variable(tf.truncated_normal([784, 512], stddev=0.1))
b1 = tf.Variable(tf.zeros([512]))


hiddle1 = tf.nn.relu(tf.matmul(x, W1) + b1)

w2 = tf.Variable(tf.truncated_normal([512, 256], stddev=0.1))
b2 = tf.Variable(tf.zeros([256]))
hiddle2 = tf.nn.relu(tf.matmul(hiddle1, w2) + b2)

w3 = tf.Variable(tf.truncated_normal([256, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]))

# 构造模型
pred = tf.nn.softmax(tf.matmul(hiddle2, w3) + b3)  # Softmax

# 损失函数：交叉墒
cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
# 使用梯度下降算法寻找最优解
# API tf.train.GradientDescentOptimizer
# TODO 创建梯度下降优化方法
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
# 初始化所有变量
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# 启动tf图
with tf.Session() as sess:
    sess.run(init)  # 初始化所有变量

    # 迭代训练
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(not_mnist.train.num_examples / batch_size)  # 计算数据集总共有多少批次数据
        # 训练训练所有批次数据
        for i in range(total_batch):
            batch_xs, batch_ys = not_mnist.train.next_batch(batch_size)
            # 运行优化操作和损失函数计算操作，获取损失值
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 计算平均损失
            avg_cost += c / total_batch
        # 打印显示
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # 测试模型
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # 计算准确度
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: not_mnist.test.images, y: not_mnist.test.labels}))
    saver.save(sess, 'my-model')

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)
Epoch: 0001 cost= 0.689215759
Epoch: 0002 cost= 0.541647487
Epoch: 0003 cost= 0.499323417
Epoch: 0004 cost= 0.470994654
Epoch: 0005 cost= 0.449189361
Epoch: 0006 cost= 0.431273469
Epoch: 0007 cost= 0.415993586
Epoch: 0008 cost= 0.402628057
Epoch: 0009 cost= 0.390746010
Epoch: 0010 cost= 0.380001067
Epoch: 0011 cost= 0.370166087
Epoch: 0012 cost= 0.361050624
Epoch: 0013 cost= 0.352548304
Epoch: 0014 cost= 0.344557996
Epoch: 0015 cost= 0.336991379
Epoch: 0016 cost= 0.329810912
Epoch: 0017 cost= 0.322972356
Epoch: 0018 cost= 0.316417484
Epoch: 0019 cost= 0.310134228
Epoch: 0020 cost= 0.304076769
Epoch: 0021 cost= 0.298213345
Epoch: 0022 cost= 0.292531178
Epoch: 0023 cost= 0.287005614
Epoch: 0024 cost= 0.281619018
Epoch: 0025 cost= 0.276371886
Optimization Finished!
Accuracy: 0.9486


In [28]:
# 三层神经网络--Tenorflow内置交叉商检验

not_mnist = NotMNIST()
# 参数
learning_rate = 0.01  # 梯度下降步长，寻找最优解的下降步长
training_epochs = 25  # 迭代轮数
batch_size = 100  # 批次训练数据集大小
display_step = 1

# tf图输入
x = tf.placeholder(tf.float32, [None, 784])  # mnist 数据集图片大小为28*28=784，placeholder为占位符
y = tf.placeholder(tf.float32, [None, 10])  # 识别A-J的数据，一共10个类别

# 设置模型权重
W1 = tf.Variable(tf.truncated_normal([784, 512], stddev=0.1))
b1 = tf.Variable(tf.zeros([512]))


hiddle1 = tf.nn.relu(tf.matmul(x, W1) + b1)

w2 = tf.Variable(tf.truncated_normal([512, 256], stddev=0.1))
b2 = tf.Variable(tf.zeros([256]))
hiddle2 = tf.nn.relu(tf.matmul(hiddle1, w2) + b2)

w3 = tf.Variable(tf.truncated_normal([256, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]))

# 构造模型
pred = tf.matmul(hiddle2, w3) + b3  # Softmax

# 损失函数：交叉墒
cost = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))
# cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
# 使用梯度下降算法寻找最优解
# API tf.train.GradientDescentOptimizer
# TODO 创建梯度下降优化方法
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
# 初始化所有变量
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# 启动tf图
with tf.Session() as sess:
    sess.run(init)  # 初始化所有变量

    # 迭代训练
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(not_mnist.train.num_examples / batch_size)  # 计算数据集总共有多少批次数据
        # 训练训练所有批次数据
        for i in range(total_batch):
            batch_xs, batch_ys = not_mnist.train.next_batch(batch_size)
            # 运行优化操作和损失函数计算操作，获取损失值
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 计算平均损失
            avg_cost += c / total_batch
        # 打印显示
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # 测试模型
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # 计算准确度
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: not_mnist.test.images, y: not_mnist.test.labels}))
    saver.save(sess, 'my-model')

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)
Epoch: 0001 cost= 0.681105026
Epoch: 0002 cost= 0.533564081
Epoch: 0003 cost= 0.492888694
Epoch: 0004 cost= 0.465632365
Epoch: 0005 cost= 0.444548411
Epoch: 0006 cost= 0.427261920
Epoch: 0007 cost= 0.412480148
Epoch: 0008 cost= 0.399523232
Epoch: 0009 cost= 0.387895431
Epoch: 0010 cost= 0.377304089
Epoch: 0011 cost= 0.367565733
Epoch: 0012 cost= 0.358547304
Epoch: 0013 cost= 0.350078151
Epoch: 0014 cost= 0.342128726
Epoch: 0015 cost= 0.334607138
Epoch: 0016 cost= 0.327446634
Epoch: 0017 cost= 0.320615263
Epoch: 0018 cost= 0.314070930
Epoch: 0019 cost= 0.307767516
Epoch: 0020 cost= 0.301704142
Epoch: 0021 cost= 0.295839207
Epoch: 0022 cost= 0.290144141
Epoch: 0023 cost= 0.284604484
Epoch: 0024 cost= 0.279198816
Epoch: 0025 cost= 0.273951143
Optimization Finished!
Accuracy: 0.9504


In [31]:
# 三层神经网络--ADAM优化器来做梯度最速下降

not_mnist = NotMNIST()
# 参数
learning_rate = 0.01  # 梯度下降步长，寻找最优解的下降步长
training_epochs = 25  # 迭代轮数
batch_size = 100  # 批次训练数据集大小
display_step = 1

# tf图输入
x = tf.placeholder(tf.float32, [None, 784])  # mnist 数据集图片大小为28*28=784，placeholder为占位符
y = tf.placeholder(tf.float32, [None, 10])  # 识别A-J的数据，一共10个类别

# 设置模型权重
W1 = tf.Variable(tf.truncated_normal([784, 512], stddev=0.1))
b1 = tf.Variable(tf.zeros([512]))


hiddle1 = tf.nn.relu(tf.matmul(x, W1) + b1)

w2 = tf.Variable(tf.truncated_normal([512, 256], stddev=0.1))
b2 = tf.Variable(tf.zeros([256]))
hiddle2 = tf.nn.relu(tf.matmul(hiddle1, w2) + b2)

w3 = tf.Variable(tf.truncated_normal([256, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]))

# 构造模型
pred = tf.matmul(hiddle2, w3) + b3  # Softmax

# 损失函数：交叉墒
cost = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))
# cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
# 使用梯度下降算法寻找最优解
# API tf.train.GradientDescentOptimizer
# TODO 创建梯度下降优化方法
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
# 初始化所有变量
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# 启动tf图
with tf.Session() as sess:
    sess.run(init)  # 初始化所有变量

    # 迭代训练
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(not_mnist.train.num_examples / batch_size)  # 计算数据集总共有多少批次数据
        # 训练训练所有批次数据
        for i in range(total_batch):
            batch_xs, batch_ys = not_mnist.train.next_batch(batch_size)
            # 运行优化操作和损失函数计算操作，获取损失值
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 计算平均损失
            avg_cost += c / total_batch
        # 打印显示
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # 测试模型
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # 计算准确度
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: not_mnist.test.images, y: not_mnist.test.labels}))
    saver.save(sess, 'my-model')

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)
Epoch: 0001 cost= 0.613398397
Epoch: 0002 cost= 0.560293347
Epoch: 0003 cost= 0.539452713
Epoch: 0004 cost= 0.532194192
Epoch: 0005 cost= 0.535462889
Epoch: 0006 cost= 0.535729321
Epoch: 0007 cost= 0.529827760
Epoch: 0008 cost= 0.534912627
Epoch: 0009 cost= 0.535853486
Epoch: 0010 cost= 0.533199989
Epoch: 0011 cost= 0.541678960
Epoch: 0012 cost= 0.527564469
Epoch: 0013 cost= 0.593428790
Epoch: 0014 cost= 0.556844672
Epoch: 0015 cost= 0.567093226
Epoch: 0016 cost= 0.537637783
Epoch: 0017 cost= 0.560684001
Epoch: 0018 cost= 0.561205780
Epoch: 0019 cost= 0.591381959
Epoch: 0020 cost= 0.540282858
Epoch: 0021 cost= 0.569012239
Epoch: 0022 cost= 0.555735716
Epoch: 0023 cost= 0.576727834
Epoch: 0024 cost= 0.580019323
Epoch: 0025 cost= 0.556027994
Optimization Finished!
Accuracy: 0.9033


In [29]:
# 三层神经网络--使用dropout


not_mnist = NotMNIST()
# 参数
learning_rate = 0.01  # 梯度下降步长，寻找最优解的下降步长
training_epochs = 25  # 迭代轮数
batch_size = 100  # 批次训练数据集大小
display_step = 1

# tf图输入
x = tf.placeholder(tf.float32, [None, 784])  # mnist 数据集图片大小为28*28=784，placeholder为占位符
y = tf.placeholder(tf.float32, [None, 10])  # 识别A-J的数据，一共10个类别

# 设置模型权重
W1 = tf.Variable(tf.truncated_normal([784, 512], stddev=0.1))
b1 = tf.Variable(tf.zeros([512]))


hiddle1 = tf.nn.relu(tf.matmul(x, W1) + b1)

# 一般只有训练的时候才使用
hiddle1 = tf.nn.dropout(hiddle1, 0.9)

w2 = tf.Variable(tf.truncated_normal([512, 256], stddev=0.1))
b2 = tf.Variable(tf.zeros([256]))
hiddle2 = tf.nn.relu(tf.matmul(hiddle1, w2) + b2)

w3 = tf.Variable(tf.truncated_normal([256, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]))

# 构造模型
pred = tf.nn.softmax(tf.matmul(hiddle2, w3) + b3)  # Softmax

# 损失函数：交叉墒
cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
# 使用梯度下降算法寻找最优解
# API tf.train.GradientDescentOptimizer
# TODO 创建梯度下降优化方法
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
# 初始化所有变量
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# 启动tf图
with tf.Session() as sess:
    sess.run(init)  # 初始化所有变量

    # 迭代训练
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(not_mnist.train.num_examples / batch_size)  # 计算数据集总共有多少批次数据
        # 训练训练所有批次数据
        for i in range(total_batch):
            batch_xs, batch_ys = not_mnist.train.next_batch(batch_size)
            # 运行优化操作和损失函数计算操作，获取损失值
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 计算平均损失
            avg_cost += c / total_batch
        # 打印显示
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # 测试模型
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # 计算准确度
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: not_mnist.test.images, y: not_mnist.test.labels}))
    saver.save(sess, 'my-model')

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)
Epoch: 0001 cost= 0.726787426
Epoch: 0002 cost= 0.569780665
Epoch: 0003 cost= 0.526560011
Epoch: 0004 cost= 0.498689446
Epoch: 0005 cost= 0.478932672
Epoch: 0006 cost= 0.462058783
Epoch: 0007 cost= 0.448072063
Epoch: 0008 cost= 0.436056945
Epoch: 0009 cost= 0.425758056
Epoch: 0010 cost= 0.416173862
Epoch: 0011 cost= 0.407189738
Epoch: 0012 cost= 0.399821637
Epoch: 0013 cost= 0.393564338
Epoch: 0014 cost= 0.385528023
Epoch: 0015 cost= 0.378683767
Epoch: 0016 cost= 0.372800741
Epoch: 0017 cost= 0.367276813
Epoch: 0018 cost= 0.361631806
Epoch: 0019 cost= 0.356756825
Epoch: 0020 cost= 0.352298431
Epoch: 0021 cost= 0.347416021
Epoch: 0022 cost= 0.342661228
Epoch: 0023 cost= 0.337509289
Epoch: 0024 cost= 0.333638692
Epoch: 0025 cost= 0.330556077
Optimization Finished!
Accuracy: 0.9423


In [6]:
# 三层神经网络--使用dropout和regularization


not_mnist = NotMNIST()
# 参数
learning_rate = 0.01  # 梯度下降步长，寻找最优解的下降步长
training_epochs = 25  # 迭代轮数
batch_size = 100  # 批次训练数据集大小
display_step = 1

# tf图输入
x = tf.placeholder(tf.float32, [None, 784])  # mnist 数据集图片大小为28*28=784，placeholder为占位符
y = tf.placeholder(tf.float32, [None, 10])  # 识别A-J的数据，一共10个类别

# 设置模型权重
W1 = tf.Variable(tf.truncated_normal([784, 512], stddev=0.1))
b1 = tf.Variable(tf.zeros([512]))


hiddle1 = tf.nn.relu(tf.matmul(x, W1) + b1)

w2 = tf.Variable(tf.truncated_normal([512, 256], stddev=0.1))
b2 = tf.Variable(tf.zeros([256]))
hiddle2 = tf.nn.relu(tf.matmul(hiddle1, w2) + b2)

w3 = tf.Variable(tf.truncated_normal([256, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]))

# 构造模型
pred = tf.matmul(hiddle2, w3) + b3  # Softmax

# 损失函数：交叉墒
cost = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))

# 一般只有训练的时候才使用
cost = cost + 5e-4 * (tf.nn.l2_loss(W1) + tf.nn.l2_loss(b1) 
                      + tf.nn.l2_loss(w2) + tf.nn.l2_loss(b2) 
                      + tf.nn.l2_loss(w3) + tf.nn.l2_loss(b3))
# cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
# 使用梯度下降算法寻找最优解
# API tf.train.GradientDescentOptimizer
# TODO 创建梯度下降优化方法
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
# 初始化所有变量
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# 启动tf图
with tf.Session() as sess:
    sess.run(init)  # 初始化所有变量

    # 迭代训练
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(not_mnist.train.num_examples / batch_size)  # 计算数据集总共有多少批次数据
        # 训练训练所有批次数据
        for i in range(total_batch):
            batch_xs, batch_ys = not_mnist.train.next_batch(batch_size)
            # 运行优化操作和损失函数计算操作，获取损失值
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 计算平均损失
            avg_cost += c / total_batch
        # 打印显示
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # 测试模型
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # 计算准确度
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: not_mnist.test.images, y: not_mnist.test.labels}))
    saver.save(sess, 'my-model')

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)
Epoch: 0001 cost= 1.715966838
Epoch: 0002 cost= 1.545387718
Epoch: 0003 cost= 1.484486620
Epoch: 0004 cost= 1.438206981
Epoch: 0005 cost= 1.398997858
Epoch: 0006 cost= 1.364090698
Epoch: 0007 cost= 1.332195905
Epoch: 0008 cost= 1.302562066
Epoch: 0009 cost= 1.274742561
Epoch: 0010 cost= 1.248429963
Epoch: 0011 cost= 1.223404843
Epoch: 0012 cost= 1.199464578
Epoch: 0013 cost= 1.176491778
Epoch: 0014 cost= 1.154372920
Epoch: 0015 cost= 1.133049127
Epoch: 0016 cost= 1.112442442
Epoch: 0017 cost= 1.092499563
Epoch: 0018 cost= 1.073184181
Epoch: 0019 cost= 1.054428055
Epoch: 0020 cost= 1.036210758
Epoch: 0021 cost= 1.018520615
Epoch: 0022 cost= 1.001314548
Epoch: 0023 cost= 0.984579798
Epoch: 0024 cost= 0.968294599
Epoch: 0025 cost= 0.952436407
Optimization Finished!
Accuracy: 0.9483


In [5]:
# 三层神经网络--使用dropout和regularization


not_mnist = NotMNIST()
# 参数
learning_rate = 0.01  # 梯度下降步长，寻找最优解的下降步长
training_epochs = 25  # 迭代轮数
batch_size = 100  # 批次训练数据集大小
display_step = 1

# tf图输入
x = tf.placeholder(tf.float32, [None, 784])  # mnist 数据集图片大小为28*28=784，placeholder为占位符
y = tf.placeholder(tf.float32, [None, 10])  # 识别A-J的数据，一共10个类别

# 设置模型权重
W1 = tf.Variable(tf.truncated_normal([784, 512], stddev=0.1))
b1 = tf.Variable(tf.zeros([512]))


hiddle1 = tf.nn.relu(tf.matmul(x, W1) + b1)

# 一般只有训练的时候才使用
hiddle1 = tf.nn.dropout(hiddle1, 0.9)

w2 = tf.Variable(tf.truncated_normal([512, 256], stddev=0.1))
b2 = tf.Variable(tf.zeros([256]))
hiddle2 = tf.nn.relu(tf.matmul(hiddle1, w2) + b2)

w3 = tf.Variable(tf.truncated_normal([256, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]))

# 构造模型
pred = tf.matmul(hiddle2, w3) + b3  # Softmax

# 损失函数：交叉墒
cost = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))

# 一般只有训练的时候才使用
cost = cost + 5e-4 * (tf.nn.l2_loss(W1) + tf.nn.l2_loss(b1) 
                      + tf.nn.l2_loss(w2) + tf.nn.l2_loss(b2) 
                      + tf.nn.l2_loss(w3) + tf.nn.l2_loss(b3))
# cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
# 使用梯度下降算法寻找最优解
# API tf.train.GradientDescentOptimizer
# TODO 创建梯度下降优化方法
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
# 初始化所有变量
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# 启动tf图
with tf.Session() as sess:
    sess.run(init)  # 初始化所有变量

    # 迭代训练
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(not_mnist.train.num_examples / batch_size)  # 计算数据集总共有多少批次数据
        # 训练训练所有批次数据
        for i in range(total_batch):
            batch_xs, batch_ys = not_mnist.train.next_batch(batch_size)
            # 运行优化操作和损失函数计算操作，获取损失值
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,
                                                          y: batch_ys})
            # 计算平均损失
            avg_cost += c / total_batch
        # 打印显示
        if (epoch + 1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch + 1), "cost=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    # 测试模型
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # 计算准确度
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: not_mnist.test.images, y: not_mnist.test.labels}))
    saver.save(sess, 'my-model')

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)
Epoch: 0001 cost= 1.750615151
Epoch: 0002 cost= 1.574152974
Epoch: 0003 cost= 1.513224420
Epoch: 0004 cost= 1.466168661
Epoch: 0005 cost= 1.428492170
Epoch: 0006 cost= 1.393421511
Epoch: 0007 cost= 1.362110652
Epoch: 0008 cost= 1.332972252
Epoch: 0009 cost= 1.305496542
Epoch: 0010 cost= 1.280182139
Epoch: 0011 cost= 1.255038129
Epoch: 0012 cost= 1.232122558
Epoch: 0013 cost= 1.208376416
Epoch: 0014 cost= 1.187858502
Epoch: 0015 cost= 1.166718278
Epoch: 0016 cost= 1.145888965
Epoch: 0017 cost= 1.126002894
Epoch: 0018 cost= 1.107722648
Epoch: 0019 cost= 1.089255278
Epoch: 0020 cost= 1.071003852
Epoch: 0021 cost= 1.053591200
Epoch: 0022 cost= 1.037248809
Epoch: 0023 cost= 1.020271828
Epoch: 0024 cost= 1.004773813
Epoch: 0025 cost= 0.988247265
Optimization Finished!
Accuracy: 0.9446
